<a href="https://colab.research.google.com/github/yabokar1/PLL/blob/main/Presidio%20entities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install presidio_analyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 49.3 MB/s eta 0:00:00


In [ ]:
pip install presidio_anonymizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.4 MB/s eta 0:00:00


In [ ]:
!python -m spacy download en_core_web_lg

2023-12-11 18:55:47.333846: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 18:55:47.333915: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 18:55:47.333955: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 18:55:47.343449: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 18:55:49.319164: W tensorflow/compiler/

In [ ]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from tabulate import tabulate

text="My name is John and my phone number is 333-111-3333 and student number is 218516112"

# Set up the engine, loads the NLP module (spaCy model by default)
# and other PII recognizers
analyzer = AnalyzerEngine()

# Call analyzer to get results
results = analyzer.analyze(text=text,
                           language='en')

# Prepare data for tabulation
table_data = [(result.entity_type, result.start, result.end, result.score) for result in results]
headers = ["Entity Type", "Start", "End", "Score"]

print(results)

# Analyzer results are passed to the AnonymizerEngine for anonymization

anonymizer = AnonymizerEngine()

anonymized_text = anonymizer.anonymize(text=text,analyzer_results=results)

# Output the original text, anonymized text, and entity details
print("\nOriginal Text:\n")
print(text)

print("\nAnonymized Text:\n")
print(anonymized_text.text)


# Print the entity scores in a table
print("\nDetected Entities and Scores:")
print(tabulate(table_data, headers=headers))

[type: PERSON, start: 11, end: 15, score: 0.85, type: PHONE_NUMBER, start: 39, end: 51, score: 0.75, type: US_BANK_NUMBER, start: 74, end: 83, score: 0.05, type: US_SSN, start: 74, end: 83, score: 0.05, type: US_PASSPORT, start: 74, end: 83, score: 0.05, type: US_DRIVER_LICENSE, start: 74, end: 83, score: 0.01]

Original Text:

My name is John and my phone number is 333-111-3333 and student number is 218516112

Anonymized Text:

My name is <PERSON> and my phone number is <PHONE_NUMBER> and student number is <US_PASSPORT>

Detected Entities and Scores:
Entity Type          Start    End    Score
-----------------  -------  -----  -------
PERSON                  11     15     0.85
PHONE_NUMBER            39     51     0.75
US_BANK_NUMBER          74     83     0.05
US_SSN                  74     83     0.05
US_PASSPORT             74     83     0.05
US_DRIVER_LICENSE       74     83     0.01


In [ ]:
from presidio_analyzer import AnalyzerEngine, PatternRecognizer, Pattern
from presidio_anonymizer import AnonymizerEngine


In [ ]:
patterns = {
    "STUDENT_ID": Pattern(name="student_id_pattern", regex=r"\b\d{9}\b", score=0.05),
    #"FACULTY_ID": Pattern(name="faculty_id_pattern", regex=r"\b\d{9}\b", score=0.8),
    "EMAIL": Pattern(name="email_pattern", regex=r"[a-zA-Z0-9._%+-]+@yorku\.ca", score=0.8),
    "PHONE_NUMBER": Pattern(name="phone_number_pattern", regex=r"\b(?:\d{3}-?\d{3}-?\d{4}|\d{10})\b", score=0.8)
}

In [ ]:
# Create custom recognizers for each entity type
custom_recognizers = {entity: PatternRecognizer(supported_entity=entity, patterns=[pattern]) for entity, pattern in patterns.items()}

In [ ]:
# Initialize the AnalyzerEngine
analyzer = AnalyzerEngine()

In [ ]:
# Add the custom recognizers to the AnalyzerEngine
for recognizer in custom_recognizers.values():
    analyzer.registry.add_recognizer(recognizer)

# Sample text to test the analyzer
text = "My name is John and my phone number is 333-111-3333 and student number is 218516112"

# Run the analyzer
results = analyzer.analyze(text=text, language="en")

print(results)

# Prepare data for tabulation
table_data = [(result.entity_type, result.start, result.end, result.score) for result in results]
headers = ["Entity Type", "Start", "End", "Score"]

# Initialize the AnonymizerEngine
anonymizer = AnonymizerEngine()

# Anonymize the text based on the analyzer results
anonymized_text = anonymizer.anonymize(text=text, analyzer_results=results)

# Output the original text, anonymized text, and entity details
print("\nOriginal Text:\n")
print(text)

print("\nAnonymized Text:\n")
print(anonymized_text.text)

# Print the entity scores in a table
print("\nDetected Entities and Scores:")
print(tabulate(table_data, headers=headers))

[type: PERSON, start: 11, end: 15, score: 0.85, type: PHONE_NUMBER, start: 39, end: 51, score: 0.8, type: US_BANK_NUMBER, start: 74, end: 83, score: 0.05, type: US_SSN, start: 74, end: 83, score: 0.05, type: STUDENT_ID, start: 74, end: 83, score: 0.05, type: US_PASSPORT, start: 74, end: 83, score: 0.05, type: US_DRIVER_LICENSE, start: 74, end: 83, score: 0.01]

Original Text:

My name is John and my phone number is 333-111-3333 and student number is 218516112

Anonymized Text:

My name is <PERSON> and my phone number is <PHONE_NUMBER> and student number is <US_PASSPORT>

Detected Entities and Scores:
Entity Type          Start    End    Score
-----------------  -------  -----  -------
PERSON                  11     15     0.85
PHONE_NUMBER            39     51     0.8
US_BANK_NUMBER          74     83     0.05
US_SSN                  74     83     0.05
STUDENT_ID              74     83     0.05
US_PASSPORT             74     83     0.05
US_DRIVER_LICENSE       74     83     0.01
